**Задача** — проанализировать базу данных.
Имеется 5 таблиц:
- books
- authors
- publishers
- ratings
- reviews

**Шаги**
1. Цели исследования
2. Исследование таблиц — вывод первых строк
3. Вывод по одному SQL-запросу для решения каждого задания
* Сколько книг вышло после 1 января 2000 года;
* Количество обзоров и среднюю оценку для каждой книги;
* Издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключим из анализа брошюры;
* Автор с самой высокой средней оценкой книг — только книги с 50 и более оценками;
* Среднее количество обзоров от пользователей, которые поставили больше 50 оценок.
Выведите результат каждого запроса в тетрадке
4. Выводы

___
# 1. Цели исследования
Cформулировать ценностное предложение для нового продукта

___
# 2. Исследование таблиц — вывод первых строк

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
import warnings; warnings.filterwarnings(action='once')


In [3]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

**Изучим содержание таблиц**

In [25]:
#пропишем функцию, чтобы сократить обьем запроса
def select(query):
    return pd.io.sql.read_sql(query, con = engine) 

In [26]:
#смотрим таблицу books
select(
    '''
    SELECT *
    FROM books
    LIMIT 3
    '''
)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135


In [27]:
#смотрим таблицу authors
select(
    '''
    SELECT *
    FROM authors
    LIMIT 3
    '''
)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie


In [28]:
#смотрим таблицу publishers
select(
    '''
    SELECT * 
    FROM publishers
    LIMIT 3
    '''
)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books


In [29]:
#смотрим таблицу ratings
select(
    '''
    SELECT *
    FROM ratings
    LIMIT 3
    '''
)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5


In [30]:
#смотрим таблицу reviews
select(
    '''
    SELECT *
    FROM reviews
    LIMIT 3
    '''
)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...


___
**Вывод**
База данных содержит информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг.

**таблица books** - данные о книгах:
* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.

**таблица authors** - данные об авторах:
* author_id — идентификатор автора;
* author — имя автора

**таблица publishers** - данные об издательствах:
* publisher_id — идентификатор издательства;
* publisher — название издательства;

**таблица ratings** - данные о пользовательских оценках книг:
* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.

**таблица reviews** - данные о пользовательских обзорах на книги:
* review_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя пользователя, написавшего обзор;
* text— текст обзора.

Лучше организовать процесс через цикл. Подставлять название таблицы в строку-запрос можно с помощью `f-строки` или метода `format()`. Саму строку-запрос можно создавать внутри цикла на каждой итерации.

___
# 3. Вывод по одному SQL-запросу для решения каждого задания
___
**Сколько книг вышло после 1 января 2000 года**

In [57]:
#извлекаем количество книг из таблицы "books", которые были опубликованы с 1 января 2000 года.
select(
    '''
    SELECT COUNT(book_id)
    FROM books
    WHERE publication_date >= '01.01.2000'
    '''
) 

,count
0,821


С даты 01.01.2000 суммарно вышло 821 книг.

___
**Количество обзоров и среднюю оценку для каждой книги**

In [62]:
# выбираем данные из нескольких таблиц, связанных между собой посредством операторов JOIN
select(
    '''
    SELECT 
        b.book_id,
        b.title, 
        COUNT(DISTINCT rv.review_id) AS count_reviews, 
        ROUND(AVG(r.rating),2) AS avg_rating
    FROM books AS b

-- LEFT JOIN для объединения таблиц "books" и "reviews" по столбцу book_id --
-- и таблиц "books" и "ratings" по столбцу book_id --
   
   LEFT JOIN reviews AS rv
    ON b.book_id = rv.book_id

    LEFT JOIN  ratings AS r
    ON b.book_id = r.book_id

-- группируем записи по столбцам book_id и title таблицы "books" --
    GROUP BY b.book_id, b.title
    
--сортируем результаты по количеству уникальных отзывов в убывающем порядке--    
    ORDER BY count_reviews DESC 

    '''
)

,book_id,title,count_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,963,Water for Elephants,6,3.98
2,734,The Glass Castle,6,4.21
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,695,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
996,808,The Natural Way to Draw,0,3.00
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,221,Essential Tales and Poems,0,4.00


Всего в базе 1000 книг по каждой из которых посчитано количество обзоров и средний рейтинг

___
**Издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключим из анализа брошюры**

In [63]:
#выборка данных о издателях, опубликовавших наибольшее количество книг с количеством страниц больше 50

select(
    '''
    SELECT 
        p.publisher,
        COUNT(book_id)
    FROM books AS b

-- объединение таблиц "books" и "publishers" по столбцу publisher_id --
    LEFT JOIN publishers AS p
    ON p.publisher_id = b.publisher_id

-- выбираем только те записи, в которых значение количества страниц (num_pages) больше 50 --
WHERE num_pages >= 50
    
--  группируем записи по столбцу publisher таблицы "publishers" --
    GROUP BY p.publisher
    
-- сортируем результаты по количеству книг в убывающем порядке (DESC) --    
    ORDER BY count DESC 
    
-- ограничиваем числа записей до одной - той, которая содержит наибольшее количество книг --    
    LIMIT 1
  '''
)
   

,publisher,count
0,Penguin Books,42


Издательство **Penguin Books** выпустили больше всего многостраничных книг - 42

___
**Автор с самой высокой средней оценкой книг — только книги с 50 и более оценками**

In [65]:
#  выбираем автора книг с наивысшим средним рейтингом из всех авторов, у которых есть как минимум 50 оценок

select(
    '''
    SELECT 
        authors.author,
        AVG(only_ratings.avg_rating) AS avg_rating
    FROM books

-- объединение таблиц - будут выбраны все записи из таблицы books -- 
-- и только те записи из таблицы ratings, которые имеют совпадающие значения поля book_id --
    LEFT JOIN (
            SELECT
                book_id,
                AVG(rating) AS avg_rating,
                COUNT(rating) AS count_rating
            FROM ratings

-- используем подзапрос (subquery), который считает средний рейтинг и количество оценок для каждой книги (группируется по book_id) --
-- Этот подзапрос выполняется первым и результаты сохраняются во временной таблице only_ratings --
            GROUP BY book_id) AS only_ratings ON only_ratings.book_id = books.book_id 
            

    LEFT JOIN authors ON authors.author_id = books.author_id
   
-- фильтруем записи, чтобы выбрать только те, у которых количество оценок (count_rating) больше или равно 50 --   
   WHERE count_rating >= 50   
    
-- группируем записи по имени автора --
    GROUP BY authors.author
    
-- сортируем записи в порядке убывания среднего рейтинга--   
    ORDER BY avg_rating DESC
    
-- ограничиваем числа записей до одной - той, которая содержит наибольший рейтинг --
    LIMIT 1
  '''
)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Автор с самой высокой средней оценкой книг, среди тех, которые получили более 50 оценок - J.K. Rowling/Mary GrandPré.

___
**Среднее количество обзоров от пользователей, которые поставили больше 50 оценок**

**Вариант 1 - через подзапрос**

In [66]:
#извлекаем среднее количество отзывов на пользователя, который сделал более 50 оценок в таблице 'ratings'

select(
    '''
    
-- используем два подзапроса 'subquery_ratings' и 'subquery_reviews', --
-- чтобы получить данные о количестве оценок и отзывов для каждого пользователя -- 
    WITH
        subquery_ratings AS (
                            SELECT
                                ratings.username,
                                COUNT(rating) AS count_ratings
                            FROM ratings
                            GROUP BY ratings.username
                            HAVING COUNT(rating) > 50),
        subquery_reviews AS (
                            SELECT
                                reviews.username,
                                COUNT(text) AS count_reviews
                            FROM reviews
                            
-- объединяем эти два подзапроса по имени пользователя и вычисляет среднее количество отзывов с помощью функции AVG() --                            
                            GROUP BY reviews.username)
        
    SELECT
        AVG(count_reviews)
        FROM subquery_ratings
  
-- возвращаем все строки из 'subquery_ratings', даже если соответствующие строки не найдены в 'subquery_reviews'--  
    LEFT JOIN subquery_reviews ON subquery_reviews.username = subquery_ratings.username
'''
)

,avg
0,24.333333


___
**Вариант 2**

In [67]:
#извлекаем среднее количество отзывов на пользователя, который сделал более 50 оценок в таблице 'ratings'

select(
    '''

-- функцией COUNT(DISTINCT) подсчитываем уникальные тексты отзывов и уникальные имена пользователей, --
-- которые оставили отзывы, и вычисляем среднее количество отзывов на пользователя с помощью деления --
    SELECT 
    COUNT(DISTINCT review.text)/ COUNT(DISTINCT review.username) AS avg_review_count
    FROM ratings AS r

-- объединяем таблицы 'ratings' и 'reviews' с помощью оператора INNER JOIN, --
-- чтобы связать отзывы с соответствующими оценками по имени пользователя --
    INNER JOIN reviews AS review
    ON r.username = review.username

-- прописываем условие фильтрации, чтобы выбрать только пользователей, которые оставили более 50 оценок --
    WHERE r.username in ( 
                    SELECT username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating_id) > 50
                    )
'''
)

,avg_review_count
0,24


Пользователи, которые поставили больше 50 оценок - в среднем выпускают 24 обзора. 

___
# 4. Выводы

В базе всего 1000 книг, из них 821 вышла с 2020 года. 

По всем книгам в базе посчитан рейтинг и отзывы. 

Больше всего многостраничных книг (от 50) выпустило издание Penguin Books. 

Автор с самой высокой средней оценкой книг, среди тех, которые получили более 50 оценок - J.K. Rowling/Mary GrandPré.

Пользователи, которые поставили больше 50 оценок - в среднем выпускают 24 обзора.

**Предложение:** расширить асортимент книг, получивших более 50 оценок, так как среди них будут самык популярные и востребованные книги.